# Time Analysis

In this notebook an analysis of time duration of each step in the inference process of the model is performed. In more detail, between two succesive frames there are two main computation stages:

1. Calculation of the action through neural network's forward pass
2. Rendering the next frames after feeding the environment with the calculated action

We desire to accelerate the forward pass of the model in order to achieve more frames per second.

In [9]:
import sys
sys.path.append('../src')
sys.path.append('../../learning/src')

import time, torch, logging
from gym_duckietown.simulator import Simulator
from big_experiment_utils.wrappers import DiscreteWrapper, DtRewardWrapper
from collections import deque

logger = logging.getLogger('gym-duckietown')
logger.setLevel(logging.WARNING)

class Timer():
    """
    Class that implements a timer fr calculating intervals
    """

    def __init__(self) :
        """Initilize times"""
        self.t0 = None
        self.total_time = 0
        self.n_laps = 0
    
    def start(self):
        """"Start timer"""
        self.n_laps += 1
        self.t0 = time.perf_counter()
    
    def stop(self):
        """Stops timer"""
        self.total_time += (time.perf_counter() - self.t0)
        self.n_laps += 1

    def reset(self):
        """Resets timer"""
        self.t0 = None
        self.total_time = 0
        self.n_laps = 0
    
    def get_average_time(self):
        """Returns average lap time"""
        return self.total_time / self.n_laps

Initially, we create the desired environment for testing and we apply that necessary wrappers to transform the action space of the environment.

In [ ]:
env = Simulator(
    seed=None,  # random seed
    map_name="loop_empty",
    max_steps=3_500,  
    domain_rand=False,
    distortion=False,
    camera_width=80,
    camera_height=60,
    draw_curve=True,
    accept_start_angle_deg=4
)
env = DtRewardWrapper(env)
env = DiscreteWrapper(env)

timer1 = Timer()
timer2 = Timer()

In [ ]:
frame = env.reset()
stacked_frames = deque([torch.zeros(size=frame.shape).unsqueeze(0)]*5,
                        maxlen=5)
if vis: env.render()
done = False
total_reward = 0
total_frames = 0
while not done:
    frame = torch.FloatTensor(frame).unsqueeze(0)
    stacked_frames.append(frame)
    state = torch.cat(tuple(stacked_frames), dim=-1)
    action = model.infer_action(state)
    next_frame, reward, done, _ = env.step(action)
    frame = next_frame
    if vis: env.render()
    total_reward += reward
    total_frames += 1
env.close()
return total_reward, total_frames